# OpenAI tests for lambda - azure function converter

## Language Agent Tree Search Strategy

The idea behind this notebook is to try to do the conversion by using the Language Agent Tree Search Strategy. The important point is to test the model by building the code and testing its results and in case things don't work, add a reflection and use it to iterate over the solution.

The inspiration for this idea comes from the models evaluated in [Code Generation on HumanEval](https://paperswithcode.com/sota/code-generation-on-humaneval) specifically on the [LATS model](https://paperswithcode.com/paper/language-agent-tree-search-unifies-reasoning).

Here's a nice explanation of the model: [LanguageAgentTreeSearch explanation](https://andyz245.github.io/LanguageAgentTreeSearch/)

Here's a fork of their repo adding Azure and Golang support: [afrancoc2000 repo](https://github.com/afrancoc2000/LanguageAgentTreeSearch)

Here's the original Dataset on HumanEval tests: [HumanEval](https://paperswithcode.com/dataset/humaneval)

### Install the necessary libraries

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [ ]:
pip install chromadb

In [ ]:
pip install tiktoken

In [ ]:
pip install python-dotenv

### Setup

Import the libraries and environment variables to gain access to the `Open API Key`

In [1]:
import os
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key=os.environ['OPENAI_API_KEY']
base_url=os.environ['OPENAI_BASE_URL']
api_version=os.environ['OPENAI_API_VERSION']

print(base_url)

https://devsquad-eastus-2.openai.azure.com/


Let's create the llm client, we can use a chat llm from langchain

In [2]:
llm = AzureChatOpenAI(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version=api_version,
    model="gpt-4",
    temperature=0
)

### Test input

Let's define the target input that we'll use to select the examples and finally to run our query

In [3]:
def list_files_in_folders(path):
    return os.listdir(path)

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [4]:
input = read_file("../go-examplesexamples/test-inputs/basic-input-1/main.go")

### Using Advanced Chain-of-Thought (CoT) Prompt

Add a system input that explains the steps the LLM needs to follow

In [15]:
system_input = '''You are an AI that only responds with Go code, NOT ENGLISH. 
You will be given a lambda function code. 
Rewrite the code without using lambda code and using a GinGonic server instead.

Use a Go code block to write your response. For example:

```go
func main() {{
        fmt.Println(\"Hello, World!\")
}}
```
'''

### Create a Prompt template
Configure prompt template with the system and user message to call the LLM.

In [16]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", system_input),
    ("human", "{input}"),
])

In [17]:
print(final_prompt.format(input=input))

System: You are an AI that only responds with Go code, NOT ENGLISH. 
You will be given a lambda function code. 
Rewrite the code without using lambda code and using a GinGonic server instead.

Use a Go code block to write your response. For example:

```go
func main() {
        fmt.Println("Hello, World!")
}
```

Human: package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
	"github.com/msft-latam-devsquad/lambda-to-azure-converter/examples/storage"
)

type SaveRequest struct {
	Id string `json:"id"`
}

type Response struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, req *SaveRequest) (*Response, error) {
	if req == nil {
		return nil, fmt.Errorf("request can't be nil")
	}
	
	azStore := storage.NewAzureStorage()
	err := azStore.Save(ctx, req.Id)
	if err != nil {
		return nil, err
	}

	message := fmt.Sprintf("request %s was successfully saved", req.Id)
	return &Response{Message: message}, nil
}

func main() {
	lambda.Start(Handl

### Use with an LLM
Now, you can connect your model to the few-shot prompt.

In [18]:
conversation = ConversationChain(llm=llm)

result = conversation.run(final_prompt.format_messages(input=input))

In [19]:
print(result)

```go
package main

import (
	"context"
	"fmt"
	"github.com/gin-gonic/gin"
	"github.com/msft-latam-devsquad/lambda-to-azure-converter/examples/storage"
	"net/http"
)

type SaveRequest struct {
	Id string `json:"id" binding:"required"`
}

type Response struct {
	Message string `json:"message"`
}

func saveHandler(c *gin.Context) {
	var req SaveRequest
	if err := c.ShouldBindJSON(&req); err != nil {
		c.JSON(http.StatusBadRequest, gin.H{"error": err.Error()})
		return
	}

	azStore := storage.NewAzureStorage()
	err := azStore.Save(context.Background(), req.Id)
	if err != nil {
		c.JSON(http.StatusInternalServerError, gin.H{"error": err.Error()})
		return
	}

	message := fmt.Sprintf("request %s was successfully saved", req.Id)
	c.JSON(http.StatusOK, &Response{Message: message})
}

func main() {
	r := gin.Default()
	r.POST("/save", saveHandler)
	r.Run() // listen and serve on 0.0.0.0:8080 (for windows "localhost:8080")
}
```


### Check if compiles 

The idea now is to programmatically check if the code is what we need so we need to create a temporal project, compile it and run unit tests to it to guarantee that the code is doing what we need.

In [28]:
root_path = "c:\\Workspace\\Americanas\\lambda-azfunction-converter-poc\\notebook"

def create_project() -> str:
    pid = os.getpid()               # get id of the process
    rand = os.urandom(8).hex()      # get random number
    
    temp_path = os.path.join(root_path, "temp")
    temp_dir = f"{temp_path}/go-lats-{pid}-{rand}"
    os.makedirs(temp_dir, exist_ok=True)
    
    # initialize a go project
    os.chdir(temp_dir)
    os.system(f"go mod init go-lats-{pid}-{rand}")
    return temp_dir

We need to write the file and fix any issues with imports, format and dependencies

In [30]:
from typing import List

def write_to_file(path: str, code: str):
    if not code.startswith("package "):
        code = f"package main\n\n{code}" 
    if os.path.exists(path):
        os.remove(path)
    with open(path, "w") as f:
        f.write(code)

def format_files(paths: List[str]):
    for path in paths:
        os.system(f"go fmt {path}")
        os.system(f"goimports -w {path}")
        os.system("go get -d ./...")
        os.system("go mod tidy")

In [32]:
temp_dir = create_project()
main_path = "main.go"

code = result.replace("```go\n", "").replace("```\n", "")
write_to_file(main_path, code)
format_files([main_path])

Now, let's check if the code compiles and let's capture the output in case of errors

In [33]:
from typing import Optional, Tuple
import subprocess

def run_process(cmd: str, tmp_path: str) -> Optional[Tuple[str, str]]:
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE, cwd=tmp_path)
    out, err = p.communicate()
    out = out.decode("utf-8")
    err = err.decode("utf-8")
    return out, err


In [34]:
buildResult = run_process("go build ./...", temp_dir)

Now we need to extract the errors as a list to be able to use them

In [35]:
class CompileErr:
    def __init__(self, rendered):
        self.rendered = rendered

    def __str__(self):
        return self.rendered

    def __repr__(self):
        return "{" + str(self) + "}"
    
def grab_compile_errs(inp: str) -> List[CompileErr]:
    # we get a stream of json objects, so we need to parse them one by one
    objs = []
    compileErr = ""
    for line in inp.splitlines():
        if line == "":
            continue
        if line.startswith("#"):
            continue
        if line.startswith(".\\lats.go"):
            if compileErr != "":
                objs.append(CompileErr(compileErr))
            compileErr = line.strip() + "\n"
        if line.startswith("        "):
            compileErr += line.strip() + "\n"
    
    if compileErr != "":
        objs.append(CompileErr(compileErr))

    return objs

In [36]:
errs = grab_compile_errs(buildResult[1])

In [37]:
print(errs)

[]


### Generate unit tests for the code

Another important feedback to know if our transformation works is going to be unit tests, even functional tests if they are available, so let's ask the LLM to generate some for us

Let's try to get some feedback about what the input is doing

In [46]:
qa_system_input = '''
You are a Go programming assistant, an AI coding assistant that generates the comments for a function 
in the code you receive, to describe what it does and how it works. you include a description of the inputs
and outputs of the function, and any other relevant information.

You return the code enclosed in a code block, and include the signature of the function.

You do not return English, only Go code.

For example:

Given the following code:
```go
func main() {{
        fmt.Println(\"Hello, World!\")
}}
```

return:
```go
// main prints Hello, World! to the console
func main() {{
```

'''

In [50]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_input),
    ("human", "{input}"),
])
conversation = ConversationChain(llm=llm)

In [51]:
purpose = conversation.run(
    qa_prompt.format_messages(input=f"Generate the comment for the handler in this code \n\n{input}"))
print(purpose)

```go
// HandleRequest takes a context and a SaveRequest, saves the request ID to Azure storage,
// and returns a Response with a success message or an error if the save operation fails.
func HandleRequest(ctx context.Context, req *SaveRequest) (*Response, error) {
```


And now with the purpose and the handler signature let's generate the tests

In [52]:
test_system_input = '''
You are a Go programming assistant, an AI coding assistant that can write unique, diverse, and intuitive 
unit tests for functions given the target code.

For example:

func signature:
/// Add three numbers together.
/// This function takes three numbers as input and returns the sum of the three numbers.
func Add3Numbers(x int, y int, z int) int {{

unit tests:
```go	
func TestAdd(t *testing.T) {{
    assert := assert.New(t)
    assert.Equal(7, Add3Numbers(2, 3+rand.Intn(1000)*0, 2))
    assert.Equal(15, Add3Numbers(5, 7, 3))
}}
```
'''

In [53]:
test_prompt = ChatPromptTemplate.from_messages([
    ("system", test_system_input),
    ("human", "{input}"),
])
conversation = ConversationChain(llm=llm)

In [55]:
tests = conversation.run(
    test_prompt.format_messages(input=purpose))
print(tests)

```go
func TestHandleRequest(t *testing.T) {
    assert := assert.New(t)
    ctx := context.Background()
    req := &SaveRequest{RequestID: "test-id"}

    // Mock the Azure storage save operation here
    // You would typically use a mocking framework or a stub for the actual implementation

    // Test for a successful save operation
    response, err := HandleRequest(ctx, req)
    assert.Nil(err)
    assert.NotNil(response)
    assert.Equal("Request saved successfully", response.Message)

    // Test for a failed save operation
    // Here you would simulate a failure in saving to Azure storage
    response, err = HandleRequest(ctx, req)
    assert.NotNil(err)
    assert.Nil(response)
}
```
